In [1]:
# potrzebne bilioteki
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Output, Input
import plotly.express as px
import pandas as pd

In [2]:
# wczytanie danych
df=pd.read_excel(r'C:\Users\natal\OneDrive\Dokumenty\AnalizaDanych\DATA\1.1.Wypadki_ogolem_smiertleni_ranni_POW.xlsx')
df_Wypadki=df.groupby(["Jednostka", "Rok"], as_index = False).sum()
# map emotions to a color
print(df_Wypadki)
c = dict(zip(df_Wypadki["Jednostka"].unique(), px.colors.qualitative.G10))

          Jednostka   Rok  Wypadki ogółem  Ofiary śmiertelne  Ranni
0      DOLNOŚLĄSKIE  2014            2466                242   3154
1      DOLNOŚLĄSKIE  2015            2280                185   2968
2      DOLNOŚLĄSKIE  2016            2324                226   2907
3      DOLNOŚLĄSKIE  2017            2183                219   2749
4      DOLNOŚLĄSKIE  2018            2148                203   2674
..              ...   ...             ...                ...    ...
107  ŚWIĘTOKRZYSKIE  2016            1365                107   1724
108  ŚWIĘTOKRZYSKIE  2017            1235                115   1529
109  ŚWIĘTOKRZYSKIE  2018            1201                127   1450
110  ŚWIĘTOKRZYSKIE  2019            1056                131   1241
111  ŚWIĘTOKRZYSKIE  2020             831                 96    953

[112 rows x 5 columns]


In [3]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    dcc.Dropdown(id='dpdn2', value=df_Wypadki['Jednostka'].unique(), multi=True,
                 options=[{'label': x, 'value': x} for x in
                          df_Wypadki.Jednostka.unique()]),
    html.Div([
        dcc.Graph(id='my-graph', figure={}, clickData=None, hoverData=None,  
                  config={
                      'staticPlot': False,     
                      'scrollZoom': True,     
                      'doubleClick': 'reset',  
                      'showTips': False,       
                      'displayModeBar': True,  
                      'watermark': True,},
                  className='' #six columns
                  ),
        #dcc.Graph(id='pie-graph', figure={}, className='six columns'),
        #dcc.Graph(id='pie-graph2', figure={}, className='six columns'),
    ]),
    html.Div([dcc.Graph(id='pie-graph', figure={}, className='six columns'),#six columns
        dcc.Graph(id='pie-graph2', figure={}, className='six columns')]) #six columns
]) 

In [4]:
@app.callback(
    Output(component_id='my-graph', component_property='figure'),
    Input(component_id='dpdn2', component_property='value')
)

def update_graph(Jednostka_chosen):
    dff = df_Wypadki[df_Wypadki.Jednostka.isin(Jednostka_chosen)]
    fig = px.line(data_frame=dff, x='Rok', y='Wypadki ogółem', color='Jednostka',
                  custom_data=['Jednostka', 'Wypadki ogółem', 'Ofiary śmiertelne', 'Ranni'],
                 title=f'Wypadki ogółem: ', color_discrete_map=c)
    fig.update_traces(mode='lines+markers')
    return fig



In [5]:
@app.callback(
    Output(component_id='pie-graph', component_property='figure'),
    Input(component_id='my-graph', component_property='hoverData'),
    Input(component_id='my-graph', component_property='clickData'),
    Input(component_id='my-graph', component_property='selectedData'),
    Input(component_id='dpdn2', component_property='value')
)
def update_side_graph(hov_data, clk_data, slct_data, Jednostka_chosen):
    if hov_data is None:
        dff2 = df_Wypadki[df_Wypadki.Jednostka.isin(Jednostka_chosen)]
        dff2 = df_Wypadki[df_Wypadki.Rok == 2014]
        print(dff2)
        fig2 = px.pie(data_frame=dff2, values='Ofiary śmiertelne', names='Jednostka',
                      title='Ofiary śmiertelne w 2014',color="emotion", color_discrete_map=c)
        return fig2
    else:
        print(f'hover data: {hov_data}')
        #print(hov_data['points'][0]['customdata'][0])
        #print(f'click data: {clk_data}')
        #print(f'selected data: {slct_data}')
        dff2 = df_Wypadki[df_Wypadki.Jednostka.isin(Jednostka_chosen)]
        hov_year = hov_data['points'][0]['x']
        dff2 = dff2[dff2.Rok == hov_year]
        fig2 = px.pie(data_frame=dff2, values='Ofiary śmiertelne', names='Jednostka',
                      title=f'Ofiary śmiertelne w: {hov_year}',color='Jednostka', color_discrete_map=c)
        return fig2
    
@app.callback(
    Output(component_id='pie-graph2', component_property='figure'),
    Input(component_id='my-graph', component_property='hoverData'),
    Input(component_id='my-graph', component_property='clickData'),
    Input(component_id='my-graph', component_property='selectedData'),
    Input(component_id='dpdn2', component_property='value')
)
def update_side_graph(hov_data, clk_data, slct_data, Jednostka_chosen):
    if hov_data is None:
        dff2 = df_Wypadki[df_Wypadki.Jednostka.isin(Jednostka_chosen)]
        dff2 = df_Wypadki[df_Wypadki.Rok == 2014]
        print(dff2)
        fig2 = px.pie(data_frame=dff2, values='Ranni', names='Jednostka',
                      title='Ranni w 2014',color='Jednostka', color_discrete_map=c)
        return fig2
    else:
        print(f'hover data: {hov_data}')
        #print(hov_data['points'][0]['customdata'][0])
        #print(f'click data: {clk_data}')
        #print(f'selected data: {slct_data}')
        dff2 = df_Wypadki[df_Wypadki.Jednostka.isin(Jednostka_chosen)]
        hov_year = hov_data['points'][0]['x']
        dff2 = dff2[dff2.Rok == hov_year]
        fig2 = px.pie(data_frame=dff2, values='Ranni', names='Jednostka',
                      title=f'Ranni w: {hov_year}',color='Jednostka', color_discrete_map=c)
        return fig2
        


In [ ]:

if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Dec/2021 17:01:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2021 17:01:21] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2021 17:01:21] "GET /_dash-layout HTTP/1.1" 200 -


               Jednostka   Rok  Wypadki ogółem  Ofiary śmiertelne  Ranni
0           DOLNOŚLĄSKIE  2014            2466                242   3154
7     KUJAWSKO-POMORSKIE  2014            1044                179   1112
14             LUBELSKIE  2014            1408                195   1667
21              LUBUSKIE  2014             717                 81    957
28           MAZOWIECKIE  2014            4385                518   5211
35           MAŁOPOLSKIE  2014            3936                234   4805
42              OPOLSKIE  2014             738                104    857
49          PODKARPACKIE  2014            1751                144   2184
56             PODLASKIE  2014             692                126    793
63             POMORSKIE  2014            2724                181   3476
70   WARMIŃSKO-MAZURSKIE  2014            1645                148   2052
77         WIELKOPOLSKIE  2014            2392                268   2808
84    ZACHODNIOPOMORSKIE  2014            1418     

127.0.0.1 - - [20/Dec/2021 17:01:22] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [20/Dec/2021 17:01:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2021 17:01:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2021 17:01:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2021 17:01:25] "POST /_dash-update-component HTTP/1.1" 200 -


hover data: {'points': [{'curveNumber': 3, 'pointNumber': 4, 'pointIndex': 4, 'x': 2018, 'y': 663, 'customdata': ['LUBUSKIE', 663, 79, 790]}]}hover data: {'points': [{'curveNumber': 3, 'pointNumber': 4, 'pointIndex': 4, 'x': 2018, 'y': 663, 'customdata': ['LUBUSKIE', 663, 79, 790]}]}



...

hover data: {'points': [{'curveNumber': 7, 'pointNumber': 6, 'pointIndex': 6, 'x': 2020, 'y': 1167, 'customdata': ['PODKARPACKIE', 1167, 132, 1240]}]}
hover data: {'points': [{'curveNumber': 7, 'pointNumber': 6, 'pointIndex': 6, 'x': 2020, 'y': 1167, 'customdata': ['PODKARPACKIE', 1167, 132, 1240]}]}


127.0.0.1 - - [20/Dec/2021 17:01:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2021 17:01:51] "POST /_dash-update-component HTTP/1.1" 200 -


hover data: {'points': [{'curveNumber': 0, 'pointNumber': 6, 'pointIndex': 6, 'x': 2020, 'y': 1578, 'customdata': ['DOLNOŚLĄSKIE', 1578, 165, 1794]}]}
hover data: {'points': [{'curveNumber': 0, 'pointNumber': 6, 'pointIndex': 6, 'x': 2020, 'y': 1578, 'customdata': ['DOLNOŚLĄSKIE', 1578, 165, 1794]}]}


127.0.0.1 - - [20/Dec/2021 17:01:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2021 17:01:53] "POST /_dash-update-component HTTP/1.1" 200 -


hover data: {'points': [{'curveNumber': 0, 'pointNumber': 5, 'pointIndex': 5, 'x': 2019, 'y': 1929, 'customdata': ['DOLNOŚLĄSKIE', 1929, 215, 2275]}]}
hover data: {'points': [{'curveNumber': 0, 'pointNumber': 5, 'pointIndex': 5, 'x': 2019, 'y': 1929, 'customdata': ['DOLNOŚLĄSKIE', 1929, 215, 2275]}]}
